### Dependencies

In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time # to be used to delay
import re
import glob
import requests
import json

In [2]:
# executable_path = {'executable_path':'resources/chromedriver'}
executable_path = {'executable_path':ChromeDriverManager().install()}

# create a new browser object by default it is firefox the default one
browser = Browser('chrome', **executable_path, headless = False) #headless true will not open browser

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 90.0.4430
[WDM] - Get LATEST driver version for 90.0.4430


[WDM] - Driver [C:\Users\gucas\.wdm\drivers\chromedriver\win32\90.0.4430.24\chromedriver.exe] found in cache


### Mass Shootings

In [3]:
# Empty dataframe to append tables
combined_msa = pd.DataFrame()

msa_url = 'https://www.gunviolencearchive.org/mass-shooting'
browser.visit(msa_url)
for i in range(81):
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(2)
    msa_df = pd.read_html(html)
    time.sleep(2)
    msa_df= msa_df[1]
    time.sleep(2)
    #If no Next button found, exit loop
    try:
        browser.click_link_by_partial_text("next")
    except:
        break
    
    # Append all dataframes
    combined_msa = combined_msa.append(msa_df)

    # Export to csv
    filepath = f"data/source/msa{i + 1}.csv"
    msa_df.to_csv(filepath, index=False)
    print(f"Successfully scraped table from page {i + 1}")

# Export combined dataframes
combined_msa.to_csv('data/combined_msa.csv', index=False)

C:\Users\gucas\anaconda3\envs\PythonData\lib\site-packages\splinter\driver\webdriver\__init__.py:493: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


Successfully scraped table from page 1
Successfully scraped table from page 2
Successfully scraped table from page 3
Successfully scraped table from page 4
Successfully scraped table from page 5
Successfully scraped table from page 6
Successfully scraped table from page 7
Successfully scraped table from page 8
Successfully scraped table from page 9
Successfully scraped table from page 10
Successfully scraped table from page 11
Successfully scraped table from page 12
Successfully scraped table from page 13
Successfully scraped table from page 14
Successfully scraped table from page 15
Successfully scraped table from page 16
Successfully scraped table from page 17
Successfully scraped table from page 18
Successfully scraped table from page 19
Successfully scraped table from page 20
Successfully scraped table from page 21
Successfully scraped table from page 22
Successfully scraped table from page 23
Successfully scraped table from page 24
Successfully scraped table from page 25
Successfu

In [4]:
# Close browser
browser.quit()

In [5]:
# Set column Incident Date to date format
combined_msa["Incident Date"] = pd.to_datetime(combined_msa["Incident Date"])
combined_msa.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,1987241,2021-04-26,Louisiana,Houma,656 Corporate Dr,0,5,View Incident View Source
1,1987235,2021-04-25,Alabama,Phenix City,1700 Highway 280 Bypass,0,4,View Incident View Source
2,1986593,2021-04-25,Arkansas,Little Rock,1900 E 6th St,1,3,View Incident View Source
3,1986770,2021-04-25,Missouri,Kansas City,E 18th St and Vine St,1,3,View Incident View Source
4,1985432,2021-04-24,Louisiana,New Orleans,100 block of Bourbon St,0,5,View Incident View Source


In [6]:
clean_msa = combined_msa.drop(['Operations'], axis =1)
clean_msa.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured
0,1987241,2021-04-26,Louisiana,Houma,656 Corporate Dr,0,5
1,1987235,2021-04-25,Alabama,Phenix City,1700 Highway 280 Bypass,0,4
2,1986593,2021-04-25,Arkansas,Little Rock,1900 E 6th St,1,3
3,1986770,2021-04-25,Missouri,Kansas City,E 18th St and Vine St,1,3
4,1985432,2021-04-24,Louisiana,New Orleans,100 block of Bourbon St,0,5


In [7]:
clean_msa = clean_msa.rename(columns = {'Incident ID':'incident_id','Incident Date':'incident_date', 'State':'state', 'City Or County':'city_county',
                         'Address':'address', '# Killed':'number_killed', '# Injured':'number_injured'})
clean_msa.head()

,incident_id,incident_date,state,city_county,address,number_killed,number_injured
0,1987241,2021-04-26,Louisiana,Houma,656 Corporate Dr,0,5
1,1987235,2021-04-25,Alabama,Phenix City,1700 Highway 280 Bypass,0,4
2,1986593,2021-04-25,Arkansas,Little Rock,1900 E 6th St,1,3
3,1986770,2021-04-25,Missouri,Kansas City,E 18th St and Vine St,1,3
4,1985432,2021-04-24,Louisiana,New Orleans,100 block of Bourbon St,0,5


In [8]:
# Check dataset
len(clean_msa)

1975

In [9]:
# Check for unique values
len(clean_msa["incident_id"].unique())

1975

In [ ]:
# # Drop duplicates
# clean_msa = clean_msa.drop_duplicates(subset = 'incident_id', keep = "first")

In [ ]:
# # Dataset after dropping duplicates
# len(clean_msa["incident_id"].unique())

In [10]:
# Export clean dataset
clean_msa.to_csv('data/clean_msa.csv', index=False)

# Google API

In [17]:
# Function to get latitude and longitude using a API call
def get_latlng(df):
    df['latitude'] = ""
    df['longitude'] = ""
    gkey = "Enter Google API Key"
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    for index, row in df.iterrows():
        street = row['address']
        city = row['city_county']
        state = row['state']
        address = f"{street}, {city}, {state}"
        params = {
            "address": address, 
            "key": gkey
    }
    #     target_address = "Seattle, Washington"
        response = requests.get(base_url, params=params)
        response_json = response.json()
        try:
            lat = response_json["results"][0]["geometry"]["location"]["lat"]
            lng = response_json["results"][0]["geometry"]["location"]["lng"]
            
            df.loc[index, 'latitude'] = lat
            df.loc[index, 'longitude'] = lng
        except:
            pass
    return df

In [18]:
final_msa = get_latlng(clean_msa)
final_msa.head(20)

,incident_id,incident_date,state,city_county,address,number_killed,number_injured,latitude,longitude
0,1987241,2021-04-26,Louisiana,Houma,656 Corporate Dr,0,5,36.1451,-96.0024
1,1987235,2021-04-25,Alabama,Phenix City,1700 Highway 280 Bypass,0,4,37.764,-122.174
2,1986593,2021-04-25,Arkansas,Little Rock,1900 E 6th St,1,3,37.7195,-122.425
3,1986770,2021-04-25,Missouri,Kansas City,E 18th St and Vine St,1,3,41.7718,-87.6728
4,1985432,2021-04-24,Louisiana,New Orleans,100 block of Bourbon St,0,5,30.703,-88.0787
5,1984807,2021-04-22,California,San Diego,500 block of J St,1,4,42.2699,-88.9628
6,1981805,2021-04-19,Texas,Houston,700 E Whitney St,2,4,34.0271,-118.351
7,1982260,2021-04-19,Georgia,Atlanta,1191 Oakland Ln SW,0,5,41.8839,-87.7085
8,1982020,2021-04-18,Alabama,Montgomery,2300 block of Day St,1,4,41.8723,-87.6937
9,1981521,2021-04-18,Michigan,Detroit,Ward Ave and W McNichols Rd,1,5,42.9121,-85.623


In [19]:
# Check dataset
len(final_msa)

1975

In [20]:
# Drop empty rows
final_msa = final_msa.dropna(how='any')

In [21]:
# Check dataset after dropping rows
len(final_msa)

1972

In [22]:
# Export final csv file
final_msa.to_csv("data/final_msa.csv")